In [1]:
import pystan
import numpy as np

In [20]:
# Set true parameter values
a = [0.8,1]
b = [0.2,0.1]
sigma = 0.2

In [22]:
# Generate data from true parameters

x = np.arange(0.01,10.01,0.01)
N = len(x)
ypred = a[0] * np.exp(-b[0] * x) + a[1] * np.exp(-b[1] * x)
y = ypred * np.exp(np.random.normal(loc=0, scale=sigma,size=N))

In [26]:
# Specify model

# data: the data that has to be given to the Stan program
# parameters: all unknown quantities to estimate
# transformed_parameters: paramters that depend on other parameters or data
# model: Here the likelihood and priors are specified. If no prior is given, uniform priors are assumed by default

example_code = """
data {
    int N;
    vector[N] x;
    vector[N] y;
}
parameters {
    vector[2] log_a;
    ordered[2] log_b;
    real<lower=0> sigma;
}
transformed parameters {
    vector<lower=0>[2] a;
    vector<lower=0>[2] b;
    a <- exp(log_a);
    b <- exp(log_b);
}
model {
    vector[N] ypred;
    ypred <- a[1]*exp(-b[1]*x) + a[2]*exp(-b[2]*x);
    y ~ lognormal(log(ypred), sigma);
    log_a ~ normal(0,1); 
    log_b ~ normal(0,1);
}
"""

example_dat = {'x':x,
              'y':y,
              'N':N}

sm = pystan.StanModel(model_code=example_code)



INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9f4a52ac6314d820783fe3ec02dcd4a0 NOW.
/usr/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp00zvx2_0/stanfit4anon_model_9f4a52ac6314d820783fe3ec02dcd4a0_5966541945880049362.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [27]:
# Fit model
fit = sm.sampling(data=example_dat, iter=1000, chains=4)
print(fit)



Inference for Stan model: anon_model_9f4a52ac6314d820783fe3ec02dcd4a0.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
log_a[1]   0.49    0.02   0.15-6.6e-3    0.5   0.54   0.56   0.58     55   1.06
log_a[2]  -1.54    0.05    0.6  -2.72  -1.92  -1.55   -1.2  -0.23    165   1.03
log_b[1]  -2.06  4.6e-3   0.05  -2.21  -2.07  -2.04  -2.02   -2.0    110   1.04
log_b[2]    0.3    0.13   1.25  -1.92  -0.81   0.57   1.22   2.43     89   1.03
sigma       0.2  1.5e-4 4.3e-3   0.19   0.19    0.2    0.2   0.21    818   1.01
a[1]       1.65    0.03    0.2   0.99   1.65   1.72   1.75   1.79     50   1.06
a[2]       0.26    0.02   0.18   0.07   0.15   0.21    0.3    0.8    109   1.05
b[1]       0.13  5.6e-4 5.9e-3   0.11   0.13   0.13   0.13   0.14    114   1.04
b[2]       2.58    0.17    3.1   0.15   0.45   1.77   3.39  11.41    344   1.01
lp__     1117